In [53]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import StratifiedKFold
fold = StratifiedKFold(n_splits=5, shuffle = True, random_state=62)

from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier

In [54]:
train_labels=pd.read_csv("trainLabels.csv")

In [55]:
df = pd.read_csv("uni_gram_byte_features__with_size.csv")

In [56]:
df.head()

,ID,0,1,2,3,4,5,6,7,8,...,f9,fa,fb,fc,fd,fe,ff,??,size,Class
0,01azqd4InC7m9JpocGv5,601905,3905,2816,3832,3345,3242,3650,3201,2965,...,3101,3211,3097,2758,3099,2759,5753,1824,4.234863,9
1,01IsoiSMh5gxyDYTl4CB,39755,8337,7249,7186,8663,6844,8420,7589,9291,...,439,281,302,7639,518,17001,54902,8588,5.538818,2
2,01jsnpXSAlgw6aPeDxrU,93506,9542,2568,2438,8925,9330,9007,2342,9107,...,2242,2885,2863,2471,2786,2680,49144,468,3.887939,9
3,01kcPWA9K2BOxQeS5Rju,21091,1213,726,817,1257,625,550,523,1078,...,485,462,516,1133,471,761,7998,13940,0.574219,1
4,01SuzwMJEIXsK7A8dQbl,19764,710,302,433,559,410,262,249,422,...,350,209,239,653,221,242,2199,9008,0.370850,8


In [57]:
from sklearn.model_selection import train_test_split

# Split the data into features (X) and target variable (y)
X = df.drop(columns=['Class'],axis=1)  # Adjust 'target_column_name' to your actual target column
y = df['Class']  # Adjust 'target_column_name' to your actual target column

# Split the data into train and test sets (e.g., 80% train, 20% test)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shape of the train and test sets to verify the split
print('Train set shape:', X_train.shape, y_train.shape)
print('Test set shape:', X_test.shape, y_test.shape)

Train set shape: (8694, 259) (8694,)
Test set shape: (2174, 259) (2174,)


In [58]:
X_train.head(2)

,ID,0,1,2,3,4,5,6,7,8,...,f8,f9,fa,fb,fc,fd,fe,ff,??,size
4264,b8KgVcy57AWMQtHEqdxo,50579,3298,2781,2579,2682,2339,2337,2122,3306,...,2781,1957,2002,2914,2531,2871,2644,13443,221704,2.524170
2457,70f4QazO1NlpXMST2LcE,7233,2207,2072,1999,2090,2001,1980,2043,2054,...,2114,2102,2024,2051,1977,2010,2090,2343,17808,1.662842


In [59]:
X1= X.drop(columns=['ID'],axis=1)
X1.head(2)

,0,1,2,3,4,5,6,7,8,9,...,f8,f9,fa,fb,fc,fd,fe,ff,??,size
0,601905,3905,2816,3832,3345,3242,3650,3201,2965,3205,...,3687,3101,3211,3097,2758,3099,2759,5753,1824,4.234863
1,39755,8337,7249,7186,8663,6844,8420,7589,9291,358,...,6536,439,281,302,7639,518,17001,54902,8588,5.538818


In [60]:
X1.shape

(10868, 258)

In [61]:
y

0        9
1        2
2        9
3        1
4        8
        ..
10863    4
10864    4
10865    4
10866    4
10867    4
Name: Class, Length: 10868, dtype: int64

In [ ]:
stack_df = X.drop(columns=["ID"], axis=1)
num_classes = len(pd.unique(y_train))
for i, j in enumerate(fold.split(X, y)):
    stack_train_X = X1.iloc[j[0]]
    stack_train_y = y.iloc[j[0]]
    stack_test_X = X1.iloc[j[1]]
    stack_test_y = y.iloc[j[1]]
    model = LGBMClassifier(learning_rate=0.025, n_estimators=850, min_child_weight=1, num_class=num_classes,
                           boosting_type="gbdt", min_child_samples=68, random_state=62, objective="multiclass",
                           metric="multi_logloss")
    model.fit(stack_train_X, stack_train_y)
    preds = model.predict_proba(stack_test_X)
    preds = pd.DataFrame(preds)
    stack_df["0"].iloc[j[1]] = preds[0]
    stack_df["1"].iloc[j[1]] = preds[1]
    stack_df["2"].iloc[j[1]] = preds[2]
    stack_df["3"].iloc[j[1]] = preds[3]
    stack_df["4"].iloc[j[1]] = preds[4]
    stack_df["5"].iloc[j[1]] = preds[5]
    stack_df["6"].iloc[j[1]] = preds[6]
    stack_df["7"].iloc[j[1]] = preds[7]
    stack_df["8"].iloc[j[1]] = preds[8]

lgbm_stack = stack_df[["0", "1", "2", "3", "4", "5", "6", "7", "8"]]
lgbm_stack.columns = ["lgbm_Prediction1", "lgbm_Prediction2", "lgbm_Prediction3", "lgbm_Prediction4",
                      "lgbm_Prediction5", "lgbm_Prediction6", "lgbm_Prediction7", "lgbm_Prediction8",
                      "lgbm_Prediction9"]


In [ ]:
stack_df = X.drop(columns=["ID"], axis=1)

In [ ]:
y_adjusted = y - 1

stack_df = X.drop(columns=["ID"], axis=1)
for i, j in enumerate(fold.split(X, y_adjusted)):
    stack_train_X = X1.iloc[j[0]]
    stack_train_y = y_adjusted.iloc[j[0]]
    stack_test_X = X1.iloc[j[1]]
    stack_test_y = y_adjusted.iloc[j[1]]
    
    model = XGBClassifier(booster="gbtree", eta=0.0975, min_child_weight=2, random_state=62,
                          objective="multi:softprob", eval_metric="logloss")
    model.fit(stack_train_X, stack_train_y)
    preds = model.predict_proba(stack_test_X)
    
    # Assign predictions to stack_df
    stack_df.loc[j[1], "0"] = preds[:, 0]
    stack_df.loc[j[1], "1"] = preds[:, 1]
    stack_df.loc[j[1], "2"] = preds[:, 2]
    stack_df.loc[j[1], "3"] = preds[:, 3]
    stack_df.loc[j[1], "4"] = preds[:, 4]
    stack_df.loc[j[1], "5"] = preds[:, 5]
    stack_df.loc[j[1], "6"] = preds[:, 6]
    stack_df.loc[j[1], "7"] = preds[:, 7]
    stack_df.loc[j[1], "8"] = preds[:, 8]

xgb_stack = stack_df[["0", "1", "2", "3", "4", "5", "6", "7", "8"]]
xgb_stack.columns = ["xgb_Prediction1", "xgb_Prediction2", "xgb_Prediction3", "xgb_Prediction4",
                     "xgb_Prediction5", "xgb_Prediction6", "xgb_Prediction7", "xgb_Prediction8",
                     "xgb_Prediction9"]


In [ ]:
stack_df = X.drop(columns=["ID"], axis=1)
for i,j in enumerate(fold.split(X,y)):
    stack_train_X = X1.iloc[j[0]]
    stack_train_y = y.iloc[j[0]]
    stack_test_X = X1.iloc[j[1]]
    stack_test_y = y.iloc[j[1]]
    model = CatBoostClassifier(verbose=0)
    model.fit(stack_train_X,stack_train_y)
    preds = model.predict_proba(stack_test_X)
    preds = pd.DataFrame(preds)
    stack_df["0"].iloc[j[1]] = preds[0]
    stack_df["1"].iloc[j[1]] = preds[1]
    stack_df["2"].iloc[j[1]] = preds[2]
    stack_df["3"].iloc[j[1]] = preds[3]
    stack_df["4"].iloc[j[1]] = preds[4]
    stack_df["5"].iloc[j[1]] = preds[5]
    stack_df["6"].iloc[j[1]] = preds[6]
    stack_df["7"].iloc[j[1]] = preds[7]
    stack_df["8"].iloc[j[1]] = preds[8]
cat_stack = stack_df[["0","1","2","3","4","5","6","7","8"]]
cat_stack.columns = ["cat_Prediction1","cat_Prediction2","cat_Prediction3","cat_Prediction4","cat_Prediction5","cat_Prediction6","cat_Prediction7","cat_Prediction8","cat_Prediction9"]

In [ ]:
xgb_stack.head(2)

,xgb_Prediction1,xgb_Prediction2,xgb_Prediction3,xgb_Prediction4,xgb_Prediction5,xgb_Prediction6,xgb_Prediction7,xgb_Prediction8,xgb_Prediction9
0,0.00036,0.000161,0.000158,0.000076,0.000086,0.000106,0.000177,0.000130,0.998747
1,0.00010,0.999663,0.000024,0.000025,0.000026,0.000034,0.000054,0.000041,0.000035


In [ ]:
cat_stack.head(2)

,cat_Prediction1,cat_Prediction2,cat_Prediction3,cat_Prediction4,cat_Prediction5,cat_Prediction6,cat_Prediction7,cat_Prediction8,cat_Prediction9
0,0.000722,0.000034,1.042790e-04,2.995358e-06,3.336983e-06,3.909250e-04,0.000053,0.000259,9.984302e-01
1,0.000155,0.999827,2.630475e-07,7.705426e-08,7.997583e-07,4.650782e-07,0.000002,0.000013,5.236201e-07


In [66]:
lgbm_stack.shape, xgb_stack.shape, cat_stack.shape

((10868, 9), (10868, 9), (10868, 9))

In [ ]:
import joblib
from sklearn.ensemble import StackingClassifier

In [ ]:
stacked_models = {
    'lgbm': lgbm_stack,
    'xgb': xgb_stack,
    'catboost': cat_stack
}

In [ ]:
from sklearn.ensemble import RandomForestClassifier  

# Combine predictions into a single DataFrame
stacked_predictions = pd.concat([lgbm_stack, xgb_stack, cat_stack], axis=1)

# Split the stacked predictions and the target variable into train and test sets
X_stacked_train, X_stacked_test, y_stacked_train, y_stacked_test = train_test_split(
    stacked_predictions, y, test_size=0.2, random_state=42
)

In [69]:
X_stacked_train

,lgbm_Prediction1,lgbm_Prediction2,lgbm_Prediction3,lgbm_Prediction4,lgbm_Prediction5,lgbm_Prediction6,lgbm_Prediction7,lgbm_Prediction8,lgbm_Prediction9,xgb_Prediction1,...,xgb_Prediction9,cat_Prediction1,cat_Prediction2,cat_Prediction3,cat_Prediction4,cat_Prediction5,cat_Prediction6,cat_Prediction7,cat_Prediction8,cat_Prediction9
4264,0.008460,0.001831,0.003916,0.001778,0.001344,1.743389e-02,0.004736,0.014786,0.945715,0.008474,...,0.967613,0.016167,0.001481,1.449646e-04,4.215209e-05,3.359740e-05,1.103238e-02,1.058929e-03,0.018910,9.511297e-01
2457,0.000012,0.000024,0.000021,0.000022,0.000005,9.997938e-01,0.000096,0.000016,0.000011,0.000200,...,0.000108,0.000071,0.000042,1.886300e-05,6.915230e-06,7.430625e-06,9.992571e-01,5.322846e-05,0.000443,1.012011e-04
393,0.000017,0.000009,0.000018,0.000019,0.000090,9.995116e-01,0.000004,0.000322,0.000009,0.000578,...,0.000248,0.005093,0.000428,1.987424e-05,3.637100e-04,7.074153e-03,9.819522e-01,1.344556e-05,0.004498,5.571883e-04
9952,0.000009,0.999955,0.000012,0.000002,0.000001,2.672759e-06,0.000008,0.000003,0.000006,0.000115,...,0.000099,0.000116,0.999848,6.043074e-07,6.212589e-07,2.670040e-06,1.581645e-06,3.134270e-06,0.000024,2.450476e-06
3037,0.000026,0.000004,0.000007,0.999816,0.000003,1.190120e-04,0.000002,0.000004,0.000018,0.000290,...,0.000136,0.000658,0.000030,1.980897e-06,9.917581e-01,2.624750e-05,6.939534e-03,2.864405e-06,0.000405,1.778932e-04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,0.000006,0.999968,0.000006,0.000001,0.000001,7.923419e-07,0.000012,0.000002,0.000003,0.000099,...,0.000060,0.000027,0.999952,1.573438e-07,3.768415e-07,5.997471e-07,3.466708e-07,5.997843e-06,0.000013,1.092647e-06
5191,0.000036,0.000054,0.000062,0.000005,0.000004,2.038011e-05,0.000017,0.000021,0.999781,0.000356,...,0.998546,0.000369,0.000027,7.461162e-05,2.638867e-06,2.721397e-06,2.672121e-04,4.221068e-05,0.000259,9.989548e-01
5390,0.000023,0.000006,0.000016,0.000009,0.000009,1.609320e-04,0.000005,0.000011,0.999760,0.000460,...,0.998072,0.000313,0.000011,3.379677e-07,6.684003e-05,2.172332e-06,1.313151e-03,9.795525e-07,0.000238,9.980547e-01
860,0.000004,0.000008,0.999917,0.000008,0.000007,1.235319e-05,0.000021,0.000012,0.000010,0.000041,...,0.000055,0.000011,0.000002,9.998177e-01,1.776135e-06,1.033472e-06,7.221632e-05,3.522774e-06,0.000062,2.827309e-05


In [70]:
X_stacked_test

,lgbm_Prediction1,lgbm_Prediction2,lgbm_Prediction3,lgbm_Prediction4,lgbm_Prediction5,lgbm_Prediction6,lgbm_Prediction7,lgbm_Prediction8,lgbm_Prediction9,xgb_Prediction1,...,xgb_Prediction9,cat_Prediction1,cat_Prediction2,cat_Prediction3,cat_Prediction4,cat_Prediction5,cat_Prediction6,cat_Prediction7,cat_Prediction8,cat_Prediction9
5820,0.000271,0.000082,0.999466,0.000015,0.000020,0.000028,0.000045,0.000032,0.000040,0.000396,...,0.000171,0.000256,0.000018,0.999546,3.238672e-06,3.569405e-06,0.000056,0.000015,0.000055,0.000047
1511,0.998488,0.000032,0.000034,0.000015,0.000211,0.000018,0.000018,0.001175,0.000009,0.973677,...,0.000880,0.923749,0.005306,0.000035,6.035688e-05,3.780430e-03,0.001594,0.000585,0.062562,0.002329
2638,0.983553,0.000280,0.000413,0.000126,0.000334,0.013063,0.000084,0.001965,0.000182,0.659649,...,0.002799,0.880074,0.001147,0.000011,1.880129e-04,1.219605e-02,0.063794,0.000079,0.038072,0.004438
5526,0.000005,0.000006,0.999854,0.000008,0.000027,0.000033,0.000024,0.000028,0.000015,0.000048,...,0.000071,0.000003,0.000003,0.999661,1.715691e-06,1.955800e-06,0.000093,0.000006,0.000205,0.000026
1713,0.000906,0.993718,0.000427,0.000117,0.000111,0.000077,0.000035,0.001822,0.002786,0.004510,...,0.012825,0.003766,0.937418,0.000666,1.972773e-03,1.868819e-03,0.002401,0.004338,0.038565,0.009004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4110,0.000007,0.000012,0.999892,0.000008,0.000010,0.000021,0.000020,0.000018,0.000013,0.000052,...,0.000053,0.000005,0.000004,0.999772,1.561953e-06,1.427919e-06,0.000138,0.000006,0.000042,0.000031
6130,0.000081,0.000203,0.000029,0.000042,0.000103,0.000033,0.999446,0.000032,0.000030,0.000651,...,0.000132,0.000070,0.000063,0.000002,1.567906e-05,1.344704e-04,0.000043,0.999600,0.000059,0.000012
9665,0.000031,0.999749,0.000036,0.000011,0.000009,0.000049,0.000048,0.000051,0.000017,0.000144,...,0.000047,0.000519,0.999197,0.000001,7.145975e-07,6.900114e-07,0.000014,0.000029,0.000148,0.000090
8523,0.000005,0.000011,0.999896,0.000008,0.000009,0.000020,0.000019,0.000018,0.000014,0.000040,...,0.000055,0.000006,0.000004,0.999762,1.679371e-06,1.495967e-06,0.000148,0.000006,0.000044,0.000028


In [68]:
from sklearn.metrics import accuracy_score
meta_model = RandomForestClassifier(n_estimators=100, random_state=42)


meta_model.fit(X_stacked_train, y_stacked_train)


joblib.dump(meta_model, 'stacked_model.pkl')


loaded_model = joblib.load('stacked_model.pkl')

stacked_predictions_test = loaded_model.predict(X_stacked_test)

accuracy = accuracy_score(y_stacked_test, stacked_predictions_test)
print("Stacked Model Accuracy (Loaded Model):", accuracy)

Stacked Model Accuracy (Loaded Model): 0.9908003679852806


In [78]:
X_train= X_train.drop(columns=["ID"],axis=1)
X_train.shape

(8694, 258)

In [80]:
X_train

,0,1,2,3,4,5,6,7,8,9,...,f8,f9,fa,fb,fc,fd,fe,ff,??,size
4264,50579,3298,2781,2579,2682,2339,2337,2122,3306,2074,...,2781,1957,2002,2914,2531,2871,2644,13443,221704,2.524170
2457,7233,2207,2072,1999,2090,2001,1980,2043,2054,2052,...,2114,2102,2024,2051,1977,2010,2090,2343,17808,1.662842
393,11144,1090,996,963,908,857,907,867,972,867,...,1260,1054,1034,1394,1146,1222,1124,5491,12172,0.801514
9952,73401,32010,31574,30327,31998,32623,31962,29910,33491,23878,...,17159,457,354,377,6644,713,9101,57877,13640,5.981445
3037,5908,394,360,538,425,354,319,351,400,367,...,374,354,368,361,354,317,369,438,58016,0.466553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5734,182522,15483,12019,11790,13888,12056,19236,11889,15293,322,...,18991,346,188,247,12893,569,16483,61940,10060,5.742188
5191,622865,3826,3074,3513,2991,3068,3355,2959,3431,2895,...,3682,2690,3258,3142,4019,2815,3734,4761,1632,4.354492
5390,85314,627,489,416,505,616,459,481,1706,547,...,573,385,370,389,638,339,401,1730,1728,0.610107
860,11381,5535,3146,3240,3353,3335,3265,3217,3350,3365,...,3168,3212,3206,3151,3163,3142,3128,7062,1490076,6.842773


In [ ]:
y_adjusted= y_train-1
y_adjusted

In [82]:
xgb_model = XGBClassifier(booster="gbtree", eta=0.0975, min_child_weight=2, random_state=62,
                          objective="multi:softprob", eval_metric="logloss")

# Train the model on your training data 
xgb_model.fit(X_train, y_adjusted)
X_test_adjusted = X_test.drop(columns=["ID"])

y_pred = xgb_model.predict(X_test_adjusted)

In [84]:
y_pred = y_pred + 1

In [85]:
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9857405703771849


In [87]:
from joblib import dump
dump(xgb_model, 'xgb_model.joblib')

['xgb_model.joblib']